In [3]:
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

# Chroma DB에서 문서 삭제
Chroma().delete_collection()

# HuggingfaceEmbedding 함수로 Open source 임베딩 모델 로드
ko_embedding = HuggingFaceEmbeddings(
    model_name="snunlp/KR-SBERT-V40K-klueNLI-augSTS",
)

# PDF 파일 로드 및 분할
loader = PyPDFLoader("data/대한민국헌법(헌법)(제00010호)(19880225).pdf")
pages = loader.load_and_split()

# 청크 크기 및 중첩 설정
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 100)
docs = text_splitter.split_documents(pages)

# ChromaDB에 청크들을 벡터 임베딩으로 저장
db = Chroma.from_documents(docs, ko_embedding)

# Chroma를 Retriever로 활용
retriever = db.as_retriever()    # Chroma DB를 Retriever로 변환
retriever.invoke("국회의원의 의무")  # "국회의원의 의무"라는 쿼리로 검색

/opt/anaconda3/envs/RAG_CODE/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[Document(id='a800f462-fc89-4f6e-8e49-6c08b28f557a', metadata={'creationdate': '2024-04-01T21:26:24+09:00', 'creator': 'PyPDF', 'moddate': '2024-04-01T21:26:24+09:00', 'page': 4, 'page_label': '5', 'producer': 'iText 2.1.7 by 1T3XT', 'source': 'data/대한민국헌법(헌법)(제00010호)(19880225).pdf', 'total_pages': 14}, page_content='법제처                                                            5                                                       국가법령정보센터\n대한민국헌법\n③국회의원의 선거구와 비례대표제 기타 선거에 관한 사항은 법률로 정한다.\n \n제42조 국회의원의 임기는 4년으로 한다.\n \n제43조 국회의원은 법률이 정하는 직을 겸할 수 없다.\n \n제44조 ①국회의원은 현행범인인 경우를 제외하고는 회기 중 국회의 동의없이 체포 또는 구금되지 아니한다.\n②국회의원이 회기 전에 체포 또는 구금된 때에는 현행범인이 아닌 한 국회의 요구가 있으면 회기 중 석방된다.\n \n제45조 국회의원은 국회에서 직무상 행한 발언과 표결에 관하여 국회 외에서 책임을 지지 아니한다.\n \n제46조 ①국회의원은 청렴의 의무가 있다.\n②국회의원은 국가이익을 우선하여 양심에 따라 직무를 행한다.'),
 Document(id='b624693b-bf9d-4ed0-a355-d44ebadcba0a', metadata={'creationdate': '2024-04-01T21:26:24+09:00', 'creator': 'PyPDF', 'moddate': '2024-04-01T21:26:24+09:00', 'page': 3, 'page_labe

### 검색 결과 개수 조정

In [4]:
# 유사 청크 1개만 반환
retriever = db.as_retriever(search_kwargs = {"k": 1})
retriever.invoke("국회의원의 의무")

[Document(id='a800f462-fc89-4f6e-8e49-6c08b28f557a', metadata={'creationdate': '2024-04-01T21:26:24+09:00', 'creator': 'PyPDF', 'moddate': '2024-04-01T21:26:24+09:00', 'page': 4, 'page_label': '5', 'producer': 'iText 2.1.7 by 1T3XT', 'source': 'data/대한민국헌법(헌법)(제00010호)(19880225).pdf', 'total_pages': 14}, page_content='법제처                                                            5                                                       국가법령정보센터\n대한민국헌법\n③국회의원의 선거구와 비례대표제 기타 선거에 관한 사항은 법률로 정한다.\n \n제42조 국회의원의 임기는 4년으로 한다.\n \n제43조 국회의원은 법률이 정하는 직을 겸할 수 없다.\n \n제44조 ①국회의원은 현행범인인 경우를 제외하고는 회기 중 국회의 동의없이 체포 또는 구금되지 아니한다.\n②국회의원이 회기 전에 체포 또는 구금된 때에는 현행범인이 아닌 한 국회의 요구가 있으면 회기 중 석방된다.\n \n제45조 국회의원은 국회에서 직무상 행한 발언과 표결에 관하여 국회 외에서 책임을 지지 아니한다.\n \n제46조 ①국회의원은 청렴의 의무가 있다.\n②국회의원은 국가이익을 우선하여 양심에 따라 직무를 행한다.')]

### 검색 방식 변경 - MMR 검색 방식

In [5]:
# lambda_mult : 관련성(유사도)과 다양성(중복 제거) 사이의 균형을 조절. 0이면 다양성만 고려, 1이면 관련성만 고려
# fetch_k : 검색할 청크의 개수
# k : 최종적으로 반환할 청크의 개수
retriever = db.as_retriever(
    search_type = "mmr",
    search_kwargs = {"lambda_mult": 0.3, "fetch_k": 10, "k": 3}
)
retriever.invoke("국회의원의 의무")

[Document(id='a800f462-fc89-4f6e-8e49-6c08b28f557a', metadata={'creationdate': '2024-04-01T21:26:24+09:00', 'creator': 'PyPDF', 'moddate': '2024-04-01T21:26:24+09:00', 'page': 4, 'page_label': '5', 'producer': 'iText 2.1.7 by 1T3XT', 'source': 'data/대한민국헌법(헌법)(제00010호)(19880225).pdf', 'total_pages': 14}, page_content='법제처                                                            5                                                       국가법령정보센터\n대한민국헌법\n③국회의원의 선거구와 비례대표제 기타 선거에 관한 사항은 법률로 정한다.\n \n제42조 국회의원의 임기는 4년으로 한다.\n \n제43조 국회의원은 법률이 정하는 직을 겸할 수 없다.\n \n제44조 ①국회의원은 현행범인인 경우를 제외하고는 회기 중 국회의 동의없이 체포 또는 구금되지 아니한다.\n②국회의원이 회기 전에 체포 또는 구금된 때에는 현행범인이 아닌 한 국회의 요구가 있으면 회기 중 석방된다.\n \n제45조 국회의원은 국회에서 직무상 행한 발언과 표결에 관하여 국회 외에서 책임을 지지 아니한다.\n \n제46조 ①국회의원은 청렴의 의무가 있다.\n②국회의원은 국가이익을 우선하여 양심에 따라 직무를 행한다.'),
 Document(id='d5b90941-2db7-4fa9-bebc-220752bb7c3d', metadata={'creationdate': '2024-04-01T21:26:24+09:00', 'creator': 'PyPDF', 'moddate': '2024-04-01T21:26:24+09:00', 'page': 2, 'page_labe

### 검색 방식 변경 - 일반 유사도 검색 방식

In [ ]:
# Chroma를 Retriever로 활용
retriever = db.as_retriever(search_kwargs = {"k": 3})
retriever.get_relevant_documents("국회의원의 의무")


/var/folders/nb/m5pjt20x3jq26zr0wyzw53zw0000gn/T/ipykernel_2101/2600206637.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents("국회의원의 의무")


[Document(id='a800f462-fc89-4f6e-8e49-6c08b28f557a', metadata={'creationdate': '2024-04-01T21:26:24+09:00', 'creator': 'PyPDF', 'moddate': '2024-04-01T21:26:24+09:00', 'page': 4, 'page_label': '5', 'producer': 'iText 2.1.7 by 1T3XT', 'source': 'data/대한민국헌법(헌법)(제00010호)(19880225).pdf', 'total_pages': 14}, page_content='법제처                                                            5                                                       국가법령정보센터\n대한민국헌법\n③국회의원의 선거구와 비례대표제 기타 선거에 관한 사항은 법률로 정한다.\n \n제42조 국회의원의 임기는 4년으로 한다.\n \n제43조 국회의원은 법률이 정하는 직을 겸할 수 없다.\n \n제44조 ①국회의원은 현행범인인 경우를 제외하고는 회기 중 국회의 동의없이 체포 또는 구금되지 아니한다.\n②국회의원이 회기 전에 체포 또는 구금된 때에는 현행범인이 아닌 한 국회의 요구가 있으면 회기 중 석방된다.\n \n제45조 국회의원은 국회에서 직무상 행한 발언과 표결에 관하여 국회 외에서 책임을 지지 아니한다.\n \n제46조 ①국회의원은 청렴의 의무가 있다.\n②국회의원은 국가이익을 우선하여 양심에 따라 직무를 행한다.'),
 Document(id='b624693b-bf9d-4ed0-a355-d44ebadcba0a', metadata={'creationdate': '2024-04-01T21:26:24+09:00', 'creator': 'PyPDF', 'moddate': '2024-04-01T21:26:24+09:00', 'page': 3, 'page_labe

### invoke()와 get_relevant_documents()의 차이점
- 둘 다 벡터 DB에서 문서를 찾아주는 기능
- invoke(): LangChain의 Runnable 시스템용 메서드. 체이닝 시스템(Runnable)에서 호환되게 사용
- get_relevant_documents(): LangChain 전통적인 메서드. 명시적으로 관련 문서를 찾을 때 사용